<a href="https://colab.research.google.com/github/fatisepah/samples/blob/main/Lenet_scratch_with_mul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import numpy as np



In [38]:
class Convolution2D:


    def __init__(self, inputs_channel, num_filters, kernel_size, padding, stride):
        # weight size: (F, C, K, K)
        # bias size: (F) 
        self.F = num_filters
        self.K = kernel_size
        self.C = inputs_channel
        
        print("FInConv=",self.F)
        print("KInConv=",self.K)
        print("CInConv=",self.C)


        self.weights = np.zeros((self.F, self.C, self.K, self.K))
        self.bias = np.zeros((self.F, 1))
        for i in range(0,self.F):
            self.weights[i,:,:,:] = np.random.normal(loc=0, scale=np.sqrt(1./(self.C*self.K*self.K)), size=(self.C, self.K, self.K))


        for f in range(self.F):
          for c in range(self.C):
            for h in range(self.K):
              for w in range(self.K):
                self.weights[f,c,h,w]=int(self.weights[f,c,h,w]*31)

        self.weights=self.weights.astype(int)

        # print("weightsShapeConvLayer=",self.weights.shape)
        # print("weightsConvLayer=",self.weights)

        self.p = padding
        self.s = stride

        # print("F(num_filters)ConvLayer=",self.F)
        # print("K(kernel_size)ConvLayer=",self.K)
        # print("C(inputs_channel)ConvLayer=",self.C)

    def zero_padding(self, inputs, size):
        w, h = inputs.shape[0], inputs.shape[1]
        new_w = 2 * size + w
        new_h = 2 * size + h
        out = np.zeros((new_w, new_h))
        out[size:w+size, size:h+size] = inputs

        

        print("outShapezeropadding=",out.shape)
        return out

    def forward(self, inputs):
        # input size: (C, W, H)
        # output size: (N, F ,WW, HH)
        # print("inputOfConvLayer=",inputs)
        # print("inputShapeOfConvLayer=",inputs.shape)


        C = inputs.shape[0]
        W = inputs.shape[1]+2*self.p
        H = inputs.shape[2]+2*self.p


        # for f in range(self.F):
        #   for c in range(self.C):
        #     for h in range(self.K):
        #       for w in range(self.K):
        #         self.weights[f,c,h,w]=int(self.weights[f,c,h,w]*31)

        self.weights=self.weights.astype(int)


        XBit=5
        sign=1
        XYInputBuf=inputs.shape
        # print("XYInputBuf",inputs.shape)
        XInputBuf= XYInputBuf[1]+2*self.p
        YInputBuf= XYInputBuf[2]+2*self.p
        print("XInputBuf",XInputBuf)
        print("YInputBuf",YInputBuf)

        #find dimention of weight buffer
        XYWeightBuf=self.weights.shape
        XWBuf= XYWeightBuf[2]
        YWBuf= XYWeightBuf[3]
        lenWeight=len(self.weights)

        ArrayMulReshape=np.array([])
        ListKeyWeight=[]
        DicWeight={}
        #/////////////////////////////////////////////////////////////////////////////////////////////////////
        
        def UpDownCounter(Enable,x,IncDec):
          IntX=int(x)
          Enable=int(Enable)
          
          if Enable != 0:
            if IntX == 1:
              if IncDec == 1:
                IntX=1
              else:
                IntX=-1
            else:
              IntX=0
          else:
            IntX=0
          return IntX
        #/////////////////////////////////////////////////////////////////////////////////////////////////////
        def DownCounter(x):
          x-=1
          return x
        #/////////////////////////////////////////////////////////////////////////////////////////////////////
        
        def BISC(In):
          # print("InBISC",In)
          # print("TypeInBISC",type(In))
          LenSc= 2 ** XBit
          
          sc=np.array([])
          for x in range(LenSc):
            sc=np.append(sc,[0])
            
          for x in range(XBit):
            i=(2 ** x)-1
            while i<LenSc:
              sc[i] = In[x]
              i+= 2 ** (x+1)
              
          sc = np.flip(sc)
          return sc
          
        #/////////////////////////////////////////////////////////////////////////////////////////////////////
        def DecimalToBinary(num,XBit):
          i = 0
          bnum = []
          Binary=''
          while num!=0:
            rem = num%2
            bnum.insert(i, rem)
            i = i+1
            num = int(num/2)
          i = i-1
          while i>=0:
            Binary=Binary+str(bnum[i])
            i = i-1
          
          while len(Binary)<XBit:
            Binary='0'+Binary
          # print("Binary:",Binary)
          
          return Binary
        #/////////////////////////////////////////////////////////////////////////////////////////////////////
        def GetDefferentialWArray(w):
          DifferentialWArray=np.array([])
          
          #sort weight 1-D
          WeightSortArray1D=np.sort(w)
          # print("WeightSortArray1D=",WeightSortArray1D)
          
          #create sorted weight array based on Differential of weights
          
          DifferentialWArray=np.append(DifferentialWArray,WeightSortArray1D[0])
          for x in range((XWBuf*YWBuf)-1):
            Differential=WeightSortArray1D[x+1]-WeightSortArray1D[x]
            DifferentialWArray=np.append(DifferentialWArray,Differential)
          # print("DifferentialWArray=",DifferentialWArray)
          
          return DifferentialWArray
          
        #/////////////////////////////////////////////////////////////////////////////////////////////////////
        
        def SortDicIndexWeight(w):
          w=np.reshape(w,(XWBuf,YWBuf))
          print("WInSortDicShape=",w.shape)
          print("XWBuf=",XWBuf)
          print("YWBuf=",YWBuf)
          for x in range(XWBuf):
            for y in range(YWBuf):
              str=x,y
              # print("str",str)
              # print("w[x,y]",w[x,y])
              DicWeight[str]=w[x,y]
              
          #sort dic of weight baes on value:dic(key,value)
          DicWSort=dict(sorted(DicWeight.items(), key=lambda item: item[1]))
          print("DicWSort=",DicWSort)
          
          for x in range(XWBuf*YWBuf):
            ListKeyWeight.append(list(DicWSort.keys())[x])
            
          # print("ListKeyWeight=",ListKeyWeight)
            
        #/////////////////////////////////////////////////////////////////////////////////////////////////////
        
        #function for doing the functional of PE in skippy based on BISC & Uo/Down counter & Down counter
        def Mul(InputBuffer,WeightBuffer):
          OutputArray=np.array([])
          print("WeightBufferBeforReshape=",WeightBuffer)
          WeightBuffer=np.reshape(WeightBuffer,(XWBuf,YWBuf))
          print("WeightBufferAfterReshape=",WeightBuffer)

          print("InputBuffershapeBefor=",InputBuffer.shape)
          InputBuffer=np.reshape(InputBuffer,(XInputBuf,YInputBuf))
          print("InputBuffershapeAfter=",InputBuffer.shape)

          #reshape WeightSortArray to 1-D
          WeightBuffer1D=WeightBuffer.reshape(-1)
          print("WeightBuffer1D=",WeightBuffer1D)
          DifferentialW=GetDefferentialWArray(WeightBuffer1D)
          
          ClockNum=0
          for i in range(len(DifferentialW)):
            ClockNum=ClockNum+abs(DifferentialW[i])
            
          ClockNum*=((XInputBuf-XWBuf)+1)*((YInputBuf-YWBuf)+1)
          # print("ClockNum=",ClockNum)
          
          SortDicIndexWeight(WeightBuffer)
          
          for x in range((XInputBuf-XWBuf)+1):
            for y in range((YInputBuf-YWBuf)+1):
              
              SelectIn=InputBuffer[x:XWBuf+x ,y:YWBuf+y]
              # print("SelectInShape=",SelectIn.shape)
              # print("SelectIn=",SelectIn)
              
              SumOut=0
              
              for a in range(XWBuf):
                for b in range(YWBuf):
                  LenDiffW= len(DifferentialW)
                  SumList=[]
                  
                  for s in range(LenDiffW):
                    SumList.append(0)

                  X=SelectIn[a,b]
                  XBinary=DecimalToBinary(X,XBit)
                    
                  SC=BISC(XBinary)
                  andisY=0
                    
                  # print("LenDiffW",LenDiffW)
                  for z in range(LenDiffW):
                    andisSC=0
                    # print("z",z)
                      
                    ValWInDiff=DifferentialW[z]
                    #print(type(ValWInDiff))
                      
                    if ValWInDiff == 0:
                      if z==0:
                        SumList[andisY]=0
                      else:
                        SumList[andisY]=SumList[andisY-1]
                      andisY+=1
                      continue						
                        
                    #instantiating the variable of sign holder 
                    #if sign=0, number is negative & sign=1, number is positive
                      
                    if ValWInDiff<0:
                      sign=0
                      ValWInDiff=abs(ValWInDiff)
                    else:
                      sign=1
                        
                    sum=0
                    while ValWInDiff>0:
                        
                      OutUDCounter=UpDownCounter(ValWInDiff,SC[(len(SC)-1)-andisSC],sign)
                      andisSC+=1
                      ValWInDiff=DownCounter(ValWInDiff)
                      sum=sum+OutUDCounter
                        
                        
                    if z==0:
                      SumList[andisY]=sum
                    else:
                      SumList[andisY]=SumList[andisY-1]+sum
                    andisY+=1

                  # print("SumList=",SumList)
                    
                  Index=ListKeyWeight.index((a,b))
                  val=SumList[Index]
                  # print("ValIndex=",val)	
                  SumOut=SumOut+val
                  if SumOut >= (2**XBit):
                    SumOut=2**XBit
                    break
                  if SumOut <= -(2**XBit):
                    SumOut=-(2**XBit)
                    break
              # print("SumOut=",SumOut)
                
              #two lines for compute the last result baesd on stochastic(divide on 2**XBit)
              SumOut=SumOut/2**XBit
              # print("SumOut=",SumOut)	
                
              OutputArray=np.append(OutputArray,SumOut)
                
          print("OutputArray=",OutputArray)
          NewOutputArray=OutputArray.reshape((XInputBuf-XWBuf)+1,(YInputBuf-YWBuf)+1)
          print("NewOutputArray=",NewOutputArray)
          print("NewOutputArrayShape=",NewOutputArray.shape)
          return NewOutputArray


        self.inputs = np.zeros((C, W, H))
        # print("self.inputsConv=",self.inputs)

        for c in range(inputs.shape[0]):
            self.inputs[c,:,:] = self.zero_padding(inputs[c,:,:], self.p)
        print("OutshapeZero_padding=",self.inputs.shape)

        for c in range(C):
          for w in range(W):
            for h in range(H):
              self.inputs[c,w,h]= int(self.inputs[c,w,h]*31)

        self.inputs= self.inputs.astype(int)

        print("inputOfConvLayer=",self.inputs)
        print("inputShapeOfConvLayer=",self.inputs.shape)

        #############  
        WW =int((W - self.K)/self.s + 1)
        print("WW=",WW)
        ##########
        HH = int((H - self.K)/self.s + 1)
        print("HH=",HH)

        feature_maps = np.zeros((self.F, WW, HH))
        print("feature_mapsShapeOfConvLayerzero=",feature_maps.shape)

        print("FInConvInForward=",self.F)


        for f in range(self.F):
          for c in range(self.C):
            print("f=",f)
            print("c=",c)

            print("inputssShape=",self.inputs[c,:,:].shape)
            print("inputs=",self.inputs[c,:,:])
            # weights=np.reshape(self.weights[f,:,:,:],(XWBuf,YWBuf))
            print("weights=",self.weights[f,c,:,:])
            feature_maps[f,:,:]=Mul(self.inputs[c,:,:],self.weights[f,c,:,:])+self.bias[f]
            
        print("feature_mapsShapeOfConvLayer=",feature_maps.shape)
        

        return feature_maps
    

    def backward(self, output_error, learning_rate):
        # print("output_errorInBackConvShape=",output_error.shape)

        C, W, H = self.inputs.shape
        # print("inputsconvvvv.shape=",self.inputs.shape)

        WW =int((W - self.K)/self.s + 1)
        # print("WW=",WW)
        ##########
        HH = int((H - self.K)/self.s + 1)
        # print("HH=",HH)

        self.weights=self.weights.astype(float)


        dx = np.zeros(self.inputs.shape)
        dw = np.zeros(self.weights.shape)
        db = np.zeros(self.bias.shape)


        F= output_error.shape[0]
        # print("F=",F,"ForOutputErrorShapeInBackConv")
        output_error=np.reshape(output_error,(F,WW,HH))
        
        # print("output_errorInBackConvShape=",output_error.shape)

        for f in range(F):
            for w in range(0, WW, self.s):
                for h in range(0, HH, self.s):
                    dw[f,:,:,:]+=output_error[f,w,h]*self.inputs[:,w:w+self.K,h:h+self.K]
                    dx[:,w:w+self.K,h:h+self.K]+=output_error[f,w,h]*self.weights[f,:,:,:]

        for f in range(F):
            db[f] = np.sum(output_error[f, :, :])

        self.weights -= learning_rate * dw
        self.bias -= learning_rate * db
        # print("conv's backward is done")
        return dx

    def extract(self):
        return {self.name+'.weights':self.weights, self.name+'.bias':self.bias}

    def feed(self, weights, bias):
        self.weights = weights
        self.bias = bias

In [39]:
class Maxpooling2D:

    def __init__(self, pool_size, stride):
        self.pool = pool_size
        self.s = stride
        # print("pool_size(pooling layer)=",self.pool)
        # print("self.s(pooling layer)=",self.s)


    def forward(self, inputs):
        self.inputs = inputs
        C, W, H = inputs.shape
        # print("InputsShape(pooling layer)=",inputs.shape)
        new_width =int((W - self.pool)/self.s + 1)
        # print("new_width(pooling layer)=",new_width)
        new_height = int((H - self.pool)/self.s + 1)
        # print("new_height(pooling layer)=",new_height)
        out = np.zeros((C, new_width, new_height))
        for c in range(C):
            for w in range(int(W/self.s)):
                for h in range(int(H/self.s)):
                    out[c, w, h] = np.max(self.inputs[c, w*self.s:w*self.s+self.pool, h*self.s:h*self.s+self.pool])
        # print("pool's forward is done")
        return out

    def backward(self,  output_error, learning_rate):
        # print("output_errorInBackMax=",output_error.shape)
        C, W, H = self.inputs.shape
        # print("C=",C,"W=",W,"H=",H,"ForOutputErrorShapeInBackMax")
        dx = np.zeros(self.inputs.shape)
        
        for c in range(C):
            for w in range(0, W, self.pool):
                for h in range(0, H, self.pool):
                    # print("selectInput=",self.inputs[c,w:w+self.pool,h:h+self.pool])
                    # print("argmaxSelectInput=",np.argmax(self.inputs[c,w:w+self.pool,h:h+self.pool]))
                    st = np.argmax(self.inputs[c,w:w+self.pool,h:h+self.pool])
                    # print("stShape=",st.shape)
                    (idx, idy) = np.unravel_index(st, (self.pool, self.pool))
                    # print("idx=",idx,"idy=",idy)
                    dx[c, w+idx, h+idy] = output_error[c,int( w/self.pool), int(h/self.pool)]
                    # print("dxshape=",dx.shape)
        # print("pool's backward is done")
        return dx

    def extract(self):
        return 

In [40]:
class FCLayer:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        # print("input_size(FC layer)=",self.input_size)
        # print("output_size(FC layer)=",self.output_size)
        self.weights = np.random.randn(input_size, output_size) / np.sqrt(input_size + output_size)
        self.bias = np.random.randn(1, output_size) / np.sqrt(input_size + output_size)

    def forward(self, input):
        self.input = input
        return np.dot(input, self.weights) + self.bias
        # print("FC's forward is done")

    def backward(self, output_error, learning_rate):
        # print("FCLayer's shape_output_error----In=",output_error.shape)
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # bias_error = output_error
        
        
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        # print("FCLayer input_errorShape=",input_error.shape)
        # print("FC's backward is done")
        return input_error

In [41]:
class ActivationLayer:
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime
    
    def forward(self, input):
        self.input = input
        return self.activation(input)
        # print("Activation's forward is done")
    
    def backward(self, output_error, learning_rate):
        return output_error * self.activation_prime(self.input)
        # print("Activation's backward is done")

In [42]:
# bonus
class FlattenLayer:
    def __init__(self, input_shape):
        self.input_shape = input_shape
        # print("input_shape(Flatten layer)=",self.input_shape)


    def forward(self, input):
        return np.reshape(input, (1, -1))
        # print("Flatten's forward is done")
    
    def backward(self, output_error, learning_rate):
        # print("Flatten's shape_output----In=",output_error.shape)
        # print("Flatten's shape_output",(np.reshape(output_error, self.input_shape)).shape)
        return np.reshape(output_error, self.input_shape)
        # print("Flatten's backward is done")

In [43]:
class Flatten:
    def __init__(self):
        pass
    def forward(self, inputs):
        self.C, self.W, self.H = inputs.shape
        return inputs.reshape(1, self.C*self.W*self.H)
    def backward(self, dy):
        return dy.reshape(self.C, self.W, self.H)
    def extract(self):
        return

In [44]:
# bonus
class SoftmaxLayer:
    def __init__(self, input_size):
        self.input_size = input_size
        # print("input_size(Softmax layer)=",self.input_size)
    
    def forward(self, input):
        self.input = input
        tmp = np.exp(input)
        self.output = tmp / np.sum(tmp)
        # print("Softmax's forward is done")
        return self.output
    
    def backward(self, output_error, learning_rate):
        # print("SoftmaxLayer's shape_output----In=",output_error.shape)
        input_error = np.zeros(output_error.shape)
        out = np.tile(self.output.T, self.input_size)
        # print("Softmax's backward is done")
        return self.output * np.dot(output_error, np.identity(self.input_size) - out)

In [45]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return np.exp(-x) / (1 + np.exp(-x))**2

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1 - np.tanh(x)**2

def relu(x):
    return np.maximum(x, 0)

def relu_prime(x):
    return np.array(x >= 0).astype('int')

In [46]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_pred.size

def sse(y_true, y_pred):
    return 0.5 * np.sum(np.power(y_true - y_pred, 2))

def sse_prime(y_true, y_pred):
    return y_pred - y_true

In [47]:
from keras.datasets import mnist
from keras.utils import np_utils

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32')
x_train /= 255
y_train = np_utils.to_categorical(y_train)
x_train = x_train[0:1000]
y_train = y_train[0:1000]

x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)

In [48]:
# unlike the Medium article, I am not encapsulating this process in a separate class
# I think it is nice just like this
network = [
    Convolution2D(1, 6, 5, 2, 1),
    ActivationLayer(relu, relu_prime),
    Maxpooling2D(2,2),
    Convolution2D(6,16,5,0,1),
    ActivationLayer(relu, relu_prime),
    Maxpooling2D(2,2),
    Convolution2D(16, 120,5,0,1),
    FlattenLayer(120),
    # FCLayer(400, 120),
    FCLayer(120, 84),
    ActivationLayer(relu, relu_prime),
    FCLayer(84, 10),
    SoftmaxLayer(10)
]

epochs = 1
learning_rate = 0.1

# training
for epoch in range(epochs):
    error = 0
    for x, y_true in zip(x_train, y_train):
        # forward
        # print("x=",x)
        # print("y_true=",y_true)
        # print("shapex=",x.shape)
        output = x.reshape((1,x.shape[0], x.shape[1]))
        # print("xReshape=",output)
        # print("shapexReshape=",output.shape)
        for layer in network:
            output = layer.forward(output)
            print("forward",layer, "finish")
            input("plz enter the word")
        
        # print("forward epoch",epoch,"finish")
        # error (display purpose only)
        error += mse(y_true, output)

        # backward
        output_error = mse_prime(y_true, output)

        for layer in reversed(network):
            output_error = layer.backward(output_error, learning_rate)
            print("backward",layer,"finish")
            # input("plz enter the word")


            # print("output_errorShape=",output_error.shape)
            

        # print("backward epoch",epoch,"finish")
        input("plz enter the word")

    
    error /= len(x_train)
    print('%d/%d, error=%f' % (epoch + 1, epochs, error))

Streaming output truncated to the last 5000 lines.
c= 3
inputssShape= (5, 5)
inputs= [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
weights= [[ 0  2  1  0  0]
 [ 0  1  2 -1 -1]
 [ 0  2 -1  3  1]
 [ 1  1  0  0  0]
 [ 1  0  0  0  1]]
WeightBufferBeforReshape= [[ 0  2  1  0  0]
 [ 0  1  2 -1 -1]
 [ 0  2 -1  3  1]
 [ 1  1  0  0  0]
 [ 1  0  0  0  1]]
WeightBufferAfterReshape= [[ 0  2  1  0  0]
 [ 0  1  2 -1 -1]
 [ 0  2 -1  3  1]
 [ 1  1  0  0  0]
 [ 1  0  0  0  1]]
InputBuffershapeBefor= (5, 5)
InputBuffershapeAfter= (5, 5)
WeightBuffer1D= [ 0  2  1  0  0  0  1  2 -1 -1  0  2 -1  3  1  1  1  0  0  0  1  0  0  0
  1]
WInSortDicShape= (5, 5)
XWBuf= 5
YWBuf= 5
DicWSort= {(1, 3): -1, (1, 4): -1, (2, 2): -1, (0, 0): 0, (0, 3): 0, (0, 4): 0, (1, 0): 0, (2, 0): 0, (3, 2): 0, (3, 3): 0, (3, 4): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (0, 2): 1, (1, 1): 1, (2, 4): 1, (3, 0): 1, (3, 1): 1, (4, 0): 1, (4, 4): 1, (0, 1): 2, (1, 2): 2, (2, 1): 2, (2, 3): 3}
OutputArray= [0.]
NewOutputArr

KeyboardInterrupt: ignored

In [ ]:
def predict(network, input):
    
    x=input.reshape(1,input.shape[0],input.shape[1])
    # print("shapereshape",(input.reshape(1,input.shape[0],input.shape[1])).shape)
    output = x
    # print("inputShape",input.shape)
    for layer in network:
        
        output = layer.forward(output)
        # print("outputShape",output.shape)
    return output

ratio = sum([np.argmax(y) == np.argmax(predict(network, x)) for x, y in zip(x_test, y_test)]) / len(x_test)
error = sum([mse(y, predict(network, x)) for x, y in zip(x_test, y_test)]) / len(x_test)
print('ratio: %.2f' % ratio)
print('mse: %.4f' % error)

In [ ]:
import matplotlib.pyplot as plt

samples = 10
for test, true in zip(x_test[:samples], y_test[:samples]):
    image = np.reshape(test, (28, 28))
    plt.imshow(image, cmap='binary')
    plt.show()
    pred = predict(network, test)[0]
    idx = np.argmax(pred)
    idx_true = np.argmax(true)
    print('pred: %s, prob: %.2f, true: %d' % (idx, pred[idx], idx_true))